In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "data_prep", ENTITY_NAME: "train_data"},
      { STEP_NAME: "data_prep", ENTITY_NAME: "eval_data"},
      # TODO: 
      { STEP_NAME: "data_prep", ENTITY_NAME: "train_eval_config"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "train_eval_data" },
        { ENTITY_NAME: "train_eval_config" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 0_cache_datasets](./imgs/0_cache_datasets.drawio.png)

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

### Loading train and val datasets and configuration files (from the previous component data_prep)

In [ ]:
from sinara.store import SinaraStore

inputs = substep.inputs(step_name = "data_prep")
outputs = substep.outputs()
tmp_outputs = substep.tmp_outputs()

# LOAD Train Images
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.train_data, tmp_dir=tmp_outputs.train_eval_data)
# LOAD Valid Images
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.eval_data, tmp_dir=tmp_outputs.train_eval_data)
# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.train_eval_config, tmp_dir=tmp_outputs.train_eval_config)

In [ ]:
#9 stop spark
SinaraSpark.stop_session()